In [1]:
import pandas as pd
import numpy as np
import gym
from keras.models import Sequential
from keras.layers import Dense
from collections import deque
import random
from IPython.display import clear_output

Using TensorFlow backend.


In [2]:
env = gym.make('MountainCar-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [12]:
class Policy():
    def __init__(self, s_size=2, a_size=3):
        self.w=1e-3*np.random.rand(s_size, a_size)
        self.a_size=a_size
    def prob(self, state):
        x = np.dot(state, self.w)
        return np.exp(x)/np.sum(np.exp(x))
    
    def act(self, state):
        prob = self.prob(state)
        action = np.argmax(prob)
        return action

In [13]:
policy = Policy()
best_w = policy.w
best_R=-np.inf
noise_scale=1e-1
num_episodes=50000
gamma=1.0
i_episode=0
scores=deque(maxlen=100)
scores.append(-np.inf)
while np.mean(scores)<-110.0:
    i_episode+=1
    rewards=[]
    state=env.reset()
    while True:
        action=policy.act(state)
        next_state, reward, done, _ = env.step(action)
        rewards.append(reward)
        state=next_state
        if done:
            break
    scores.append(np.sum(rewards))
    discounts=[gamma**i for i in range(len(rewards)+1)]
    R=np.sum([a*b for a,b in zip(rewards, discounts)])
    if R>=best_R:
        best_R=R
        best_w=policy.w
        noise_scale = max(1e-3, noise_scale/2)
        policy.w += noise_scale*np.random.rand(*policy.w.shape)
    else:
        noise_scale = min(2, noise_scale*2)
        policy.w = best_w + noise_scale*np.random.rand(*policy.w.shape)
    clear_output(True)
    print('Episode : {}/{} ... score : {}'.format(i_episode, num_episodes, np.mean(scores))) 

Episode : 2572/50000 ... score : -200.0


KeyboardInterrupt: 